In [ ]:
# 1. TOTAL SYSTEM WIPE (Prevents 'Port in Use' errors)
print("🧹 [1/5] Wiping old sessions...")
!pkill -9 dockerd || true
!pkill -9 cloudflared || true
!pkill -9 npx || true
!rm -rf /var/run/docker.pid

# 2. INSTALL ENGINE
print("🏗️ [2/5] Installing Docker Binary...")
!apt-get update -qq > /dev/null
!apt-get install -y docker.io adb -qq > /dev/null

# 3. THE MAN-IN-THE-MIDDLE START (Direct Binary Execution)
print("⚡ [3/5] Force-Starting Motor...")
import subprocess, time, os, threading
# This launches dockerd directly, ignoring 'service' and 'systemctl'
subprocess.Popen(['dockerd', '--storage-driver=vfs', '--host=unix:///var/run/docker.sock'], 
                 stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(15)

# 4. LAUNCH ANDROID & PORTAL
print("📱 [4/5] Booting Android Engine...")
!docker run -d --privileged -p 5555:5555 redroid/redroid:11.0.0-latest
!npx -y ws-scrcpy-docker --port 7860 > /dev/null 2>&1 &

# 5. GENERATE PERMANENT-STYLE LINK
print("🔗 [5/5] Generating Secure Link...")
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared

def run_tunnel():
    os.system("./cloudflared tunnel --url http://localhost:7860")

threading.Thread(target=run_tunnel, daemon=True).start()

time.sleep(20)
print("\n✅ STATUS: SUCCESSFUL")
print("👉 LOOK IN THE LOGS BELOW FOR YOUR .trycloudflare.com LINK")

# KEEP ALIVE LOOP
while True:
    time.sleep(60)
    print("✨ Connection Active...")